## Laboratory 3 - Bayesian Inference ##

# Exercise 1 #

The number of particles emitted by a radioactive source during a fixed interval of time
(∆t = 10 s) follows a Poisson distribution on the parameter µ. The number of particles
observed during consecutive time intervals is: 4, 1, 3, 1, 5 and 3. 

(a) assuming a positive uniform prior distribution for the parameter µ
- determine and draw the posterior distribution for µ, given the data
- evaluate mean, median and variance, both analytically and numerically in R 

(b) assuming a Gamma prior such that the expected value is µ = 3 with a standard
deviation σ = 1,
- determine and draw the posterior distribution for µ, given the data
- evaluate mean, median and variance, both analytically and numerically in R. 

(c) evaluate a 95% credibility interval for the results obtained with different priors. Compare the result with that obtained using a normal approximation for the posterior
distribution, with the same mean and standard deviation
